
# Titel

## Table of Contents

- Objectiv

## Objectiv

## Data (Jesse)

- adjusted close values
- N = 40 stocks
- $t_{min} = 2007  \, ,\quad t_{max} = 2017$

### What are adjusted close values


### volatility


## helper functions

In [1]:
1+1

2

## Models

- CNN
- Fully connected
- RNN 
    - LSTM 
    - GRU

- Two inputs all above

### Was ist zu erwarten...

### Aufbau

## Results (Sebastian)

### Auswertungsfunktionen

## Stuff